<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myProspects_Upside_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/quantvesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import the function from the script
import myProspects_Upside_Analysis_v2

all = 1

# Call the function to get the DataFrame
df_prospects_features = myProspects_Upside_Analysis_v2.get_prospects_features_v2(all)
cols = ['Symbol', 'Close', 'FTT',  'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'Today%', 'FTT%', 'ATH%','Gained%', 'Criteria', 'Strategy', 'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ', 'ROE%', 'CFO_2_EBITDA%']
df_prospects_features = df_prospects_features[cols].drop_duplicates()

Run date time (IST): 2025-05-03 08:34:39
-------------------


ERROR:yfinance:$ISEC.NS: possibly delisted; no price data found  (period=1mo)


qualified stocks: 258
with latest results: 243
still star stocks: 134


In [3]:
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'CapType', 'Category']

tmp_df = df_prospects_features
list_x40 = list(tmp_df[tmp_df['MBQ'].str.contains('X40', na=False)]['Symbol'].values)
list_x5k = list(tmp_df[tmp_df['MBQ'].str.contains('X5K', na=False)]['Symbol'].values)
list_x40n = list(tmp_df[tmp_df['MBQ'].str.contains('X40N', na=False)]['Symbol'].values)
list_hc = list(tmp_df[tmp_df['Conviction'].isin(['X-LC', 'H-LC', 'X-MC', 'X-SC'])]['Symbol'].values)
#len(set(list_x40 + list_x5k) - set( list_x40n))
stock_list = list(set(list_x40 + list_x5k + list_hc))

In [4]:
# High quality (X40 + X40N + X5K + X-LC + H-LC + X-MC + X-SC) prospects
df_tmp = df_prospects_features[df_prospects_features['Symbol'].isin(stock_list)]

df_tmp['Rnk_Dev%_200'] = df_tmp['Dev%_200'].rank(method='min', ascending=True)
df_tmp['Rnk_Dev%_PE'] = df_tmp['Dev%_PE'].rank(method='min', ascending=True)
df_tmp['Rnk_Gained%'] = df_tmp['Gained%'].rank(method='min', ascending=True)
df_tmp['Rnk_ATH%'] = df_tmp['ATH%'].rank(method='max', ascending=False)
df_tmp['Rnk_Sales_Grwth%'] = df_tmp['Sales_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_Profit_Grwth%'] = df_tmp['Profit_Grwth%'].rank(method='max', ascending=False)
df_tmp['Rnk_MCap'] = df_tmp['MCap'].rank(method='max', ascending=False)
df_tmp['Rnk_ROE%'] = df_tmp['ROE%'].rank(method='max', ascending=False)
df_tmp['Rnk_CFO_2_EBITDA%'] = df_tmp['CFO_2_EBITDA%'].rank(method='max', ascending=False)
df_tmp['Ovrl_Rank'] = df_tmp['Rnk_Dev%_200'] + df_tmp['Rnk_Dev%_PE'] + df_tmp['Rnk_Gained%'] + df_tmp['Rnk_ATH%'] + df_tmp['Rnk_Sales_Grwth%'] + df_tmp['Rnk_Profit_Grwth%'] + df_tmp['Rnk_MCap'] + df_tmp['Rnk_CFO_2_EBITDA%'] + df_tmp['Rnk_ROE%']
df_tmp['Cuml_Rank'] = df_tmp['Ovrl_Rank'].rank(method='min', ascending=True)
cols = ['Symbol', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP', 'FTT%', 'ATH%','Gained%', 'Cuml_Rank', 'Strategy', 'Category', 'InFolio']
data_table.DataTable(df_tmp[cols].sort_values(by = 'Cuml_Rank', ascending=True), include_index=False)

,Symbol,FTT,Dev%_200,Dev%_PE,Conviction,RSP,FTT%,ATH%,Gained%,Cuml_Rank,Strategy,Category,InFolio
319,TATAMOTORS,1065.00,-21.89,-56.95,M-LC,20.43,64.07,78.99,11.96,1.0,NTT,AUTO,1.0
4,ABB,8953.12,-19.96,-40.07,H-LC,58.75,64.55,64.55,10.92,2.0,ATH,ELECTRICAL,1.0
100,DLF,1031.70,-12.48,-25.51,H-LC,42.41,50.55,50.55,12.70,3.0,ATH,REALTY,1.0
196,LICI,983.00,-13.70,-20.17,H-LC,28.40,24.08,49.97,8.64,4.0,NTT,INSURANCE,1.0
322,TCS,4998.00,-15.35,-17.76,X-LC,23.35,45.67,32.09,6.17,5.0,BTT,IT,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,EQUITASBNK,113.10,-8.93,NaN,NaN,93.39,77.41,77.41,16.10,NaN,ATH,BANKS,NaN
157,INDIGO,5558.50,17.03,44.69,M-LC,68.68,4.45,4.45,38.26,NaN,ATH,AIRLINES,NaN
168,ISEC,892.25,0.03,NaN,NaN,NaN,0.00,0.00,0.00,NaN,ATH,FINANCE,NaN
228,NESTLEIND,2755.00,-0.31,-11.36,X-LC,72.76,18.15,17.42,9.16,NaN,NTT,FMCG,1.0


In [5]:
df_tmp.columns

Index(['Symbol', 'Close', 'FTT', 'Dev%_200', 'Dev%_PE', 'Conviction', 'RSP',
       'Today%', 'FTT%', 'ATH%', 'Gained%', 'Criteria', 'Strategy',
       'Sales_Grwth%', 'Profit_Grwth%', 'Category', 'InFolio', 'MCap', 'MBQ',
       'ROE%', 'CFO_2_EBITDA%', 'Rnk_Dev%_200', 'Rnk_Dev%_PE', 'Rnk_Gained%',
       'Rnk_ATH%', 'Rnk_Sales_Grwth%', 'Rnk_Profit_Grwth%', 'Rnk_MCap',
       'Rnk_ROE%', 'Rnk_CFO_2_EBITDA%', 'Ovrl_Rank', 'Cuml_Rank'],
      dtype='object')

In [6]:
# x24/x25
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XY24', 'X25'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
254,QUESS,339.35,986.0,-48.58,NaN,NaN,0.39,2.52,190.56,252.80,...,XY24,NTT,NaN,NaN,MISC,1.0,NaN,XY24,NaN,NaN
48,BANDHANBNK,161.81,400.0,-5.92,-37.04,H-SC,78.99,-2.30,147.20,340.65,...,XY24,NTT,16.32,23.14,BANKS,1.0,26688.86,XY24,11.94,105.17
128,GREENPANEL,230.01,537.0,-31.54,151.29,M-SC,40.86,-1.01,133.47,160.73,...,XY24,NTT,26.90,17.12,MISC,1.0,3688.66,XY24,10.69,50.26
269,REPCOHOME,390.50,880.0,-12.05,-58.27,H-SC,94.94,-3.20,125.35,115.93,...,XY24,NTT,6.73,9.55,FINANCE,1.0,2133.64,XY24,14.59,-49.30
103,EASEMYTRIP,11.84,26.4,-26.00,19.86,M-SC,25.29,-1.58,122.97,188.85,...,XY24,NTT,57.00,57.31,TRAVEL,1.0,4256.51,XY24,31.90,47.72
7,ACC,1864.50,3906.0,-14.67,-36.77,M-MC,12.84,-1.24,109.49,48.69,...,XY24,BTT,9.04,7.86,CEMENT,1.0,35370.74,XY24,13.33,41.37
326,TITAGARH,741.85,1548.0,-33.82,-26.16,H-SC,5.45,-0.70,108.67,150.57,...,XY24,NTT,3.69,14.14,ENGINEERING,1.0,10044.70,XY24,18.13,17.54
302,STARHEALTH,377.05,761.0,-23.05,-16.46,M-SC,90.27,-3.27,101.83,140.51,...,XY24,NTT,14.83,-23.58,INSURANCE,1.0,22944.19,XY24,9.57,160.11
275,SAIL,113.08,228.0,-6.55,71.33,M-MC,14.01,-0.55,101.63,77.41,...,XY24,BTT,-5.10,-13.50,STEEL,1.0,46960.06,XY24,6.44,23.24
289,SFL,662.60,1287.0,-23.49,278.48,H-SC,12.06,2.84,94.23,197.65,...,XY24,NTT,56.70,17.83,MISC,1.0,7924.20,XY24/X40N,6.09,106.57


In [7]:
# watchlist
df_tmp = df_prospects_features[df_prospects_features['InFolio'] == -1].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%


In [8]:
# X-MC and X-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('X-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['X-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
11,ALKYLAMINE,1713.90,4567.52,-11.70,-17.47,X-SC,34.24,0.66,166.50,166.50,...,SR,ATH,3.07,12.23,CHEMICALS,1.0,8716.59,SR,12.29,103.01
344,VAIBHAVGBL,220.54,521.00,-20.95,43.25,X-SC,16.34,0.64,136.24,322.49,...,XR,NTT,38.45,22.42,JEWELLERY,1.0,4006.89,XR,10.76,93.80
79,CERA,5350.50,10420.45,-27.23,73.94,X-SC,9.34,-0.12,94.76,94.76,...,X40N,ATH,41.90,21.49,CERAMICS,1.0,7277.21,X40N,19.05,64.28
333,TTKPRESTIG,611.60,1149.24,-24.12,-12.42,X-SC,23.74,0.83,87.91,87.91,...,X40N,ATH,0.80,-8.02,DURABLES,1.0,8398.33,X40N,11.19,76.01
28,ASTRAL,1317.30,2412.74,-22.47,-27.98,X-MC,30.74,-2.07,83.16,83.16,...,AR,ATH,4.59,-7.52,PLASTICS,NaN,36106.19,AR,18.05,87.36
217,METROPOLIS,1698.40,3102.00,-13.44,-5.84,X-SC,89.88,-0.18,82.64,102.06,...,XR,NTT,13.60,22.02,HEALTHCARE,NaN,8741.64,XR,12.20,89.19
148,HONAUT,34680.00,58518.52,-19.79,-29.04,X-MC,49.03,0.46,68.74,68.74,...,X40N,ATH,1.73,14.34,ELECTRICAL,1.0,30458.77,X40N,14.76,59.48
96,DEEPAKNTR,1983.20,3102.60,-21.33,14.20,X-SC,27.24,2.46,56.44,56.44,...,AR,ATH,9.46,-12.67,CHEMICALS,NaN,26310.04,AR,16.38,75.09
318,TATAELXSI,5820.00,9000.00,-12.27,-22.37,X-MC,91.44,0.94,54.64,80.40,...,X40N,NTT,4.98,-0.81,IT,1.0,35910.23,X40N,29.26,70.47
91,DABUR,479.40,735.00,-12.84,-15.10,X-MC,21.79,-1.71,53.32,38.53,...,XY24,BTT,2.29,0.34,FMCG,1.0,86125.76,XY24/X40,19.22,69.88


In [9]:
# H-MC and H-SC prospects
df_tmp = df_prospects_features[df_prospects_features['Conviction'].str.startswith('H-', na=False)]
df_tmp = df_tmp[~df_tmp['Conviction'].isin(['H-LC'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
311,TANLA,498.55,1963.11,-28.68,-56.59,H-SC,73.15,5.10,293.76,293.76,...,AR,ATH,25.33,32.45,IT,1.0,6540.56,AR,31.71,75.86
261,RELAXO,407.15,1176.00,-37.17,-44.04,H-SC,20.43,-0.65,188.84,249.61,...,X40N,NTT,-3.07,-13.28,FOOTWEAR,1.0,10184.51,X40N,10.49,53.40
348,VALIANTORG,313.60,838.00,-9.89,-291.52,H-SC,10.12,-2.67,167.22,490.13,...,XR,NTT,20.45,21.02,CHEMICALS,1.0,761.26,XR,-0.91,184.05
48,BANDHANBNK,161.81,400.00,-5.92,-37.04,H-SC,78.99,-2.30,147.20,340.65,...,XY24,NTT,16.32,23.14,BANKS,1.0,26688.86,XY24,11.94,105.17
123,GLAND,1404.00,3442.00,-18.42,-17.34,H-SC,5.06,0.29,145.16,204.29,...,XR,NTT,16.63,1.59,PHARMA,NaN,23050.26,XR,9.26,66.33
355,VIPIND,307.35,718.00,-28.19,-729.00,H-SC,89.11,0.46,133.61,140.78,...,X40N,NTT,70.88,13.19,MISC,1.0,4607.81,X40N,5.25,-63.51
269,REPCOHOME,390.50,880.00,-12.05,-58.27,H-SC,94.94,-3.20,125.35,115.93,...,XY24,NTT,6.73,9.55,FINANCE,1.0,2133.64,XY24,14.59,-49.30
1,5PAISA,364.05,810.00,-19.99,123.56,H-SC,38.13,-1.99,122.50,102.18,...,X40N,BTT,39.09,9.46,FINANCE,1.0,1148.55,X40N,10.29,-85.27
159,INDUSINDBK,856.45,1800.00,-22.21,-34.14,H-MC,97.67,2.15,110.17,126.91,...,XR,NTT,15.29,-16.40,BANKS,1.0,65292.51,XR/X5K,15.25,-44.91
326,TITAGARH,741.85,1548.00,-33.82,-26.16,H-SC,5.45,-0.70,108.67,150.57,...,XY24,NTT,3.69,14.14,ENGINEERING,1.0,10044.70,XY24,18.13,17.54


In [10]:
# x40
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
358,WHIRLPOOL,1235.70,2270.00,-27.82,-37.69,M-SC,95.72,-1.88,83.70,114.59,...,X40,NTT,13.01,63.38,DURABLES,1.0,15908.32,X40,6.36,107.69
23,ASIANPAINT,2410.00,4250.00,-9.02,-28.62,X-LC,66.54,-0.65,76.35,45.13,...,X40,BTT,-3.58,-20.06,PAINTS,1.0,232357.09,X40,31.44,72.63
54,BATAINDIA,1196.80,2096.00,-12.32,-33.96,M-SC,18.87,-0.96,75.13,81.79,...,X40,NTT,1.13,23.00,FOOTWEAR,1.0,15499.13,X40,19.76,52.77
42,BAJAJ-AUTO,7820.50,12666.40,-16.60,35.39,X-LC,24.51,-2.61,61.96,61.96,...,X40,ATH,18.13,1.79,AUTO,NaN,224315.23,X40/X5K,26.48,62.56
165,INFY,1506.90,2275.00,-16.38,-16.01,X-LC,25.68,0.45,50.97,32.70,...,X40,BTT,6.06,1.89,IT,1.0,623072.66,X40/X5K,28.83,83.90
88,COLPAL,2538.00,3808.06,-14.43,3.62,X-MC,82.10,-1.91,50.04,50.04,...,X40,ATH,9.50,14.59,FMCG,1.0,70228.72,X40,74.52,60.63
242,PFIZER,4325.80,6318.30,-13.93,-13.77,H-SC,79.38,0.77,46.06,46.06,...,X40,ATH,0.76,21.18,PHARMA,1.0,19602.83,X40,16.10,31.36
322,TCS,3431.00,4998.00,-15.35,-17.76,X-LC,23.35,-0.66,45.67,32.09,...,X40,BTT,5.99,4.14,IT,1.0,1241234.83,X40/X5K,52.38,68.57
135,HAVELLS,1561.60,2077.12,-8.20,-5.75,X-LC,61.48,-2.46,33.01,33.01,...,X40,ATH,17.15,16.40,ELECTRICAL,1.0,100439.23,X40,18.65,62.25
122,GILLETTE,7932.00,10330.69,-8.31,-8.62,H-SC,29.57,-1.30,30.24,30.24,...,X40,ATH,9.74,21.21,FMCG,NaN,26109.54,X40,42.49,77.12


In [11]:
# x40n
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['X40N'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
256,RAJESHEXPO,180.76,583.00,-24.05,1655.05,M-SC,7.98,-0.33,222.53,425.20,...,X40N,NTT,17.04,8.52,JEWELLERY,1.0,5224.15,X40N,2.24,55.42
261,RELAXO,407.15,1176.00,-37.17,-44.04,H-SC,20.43,-0.65,188.84,249.61,...,X40N,NTT,-3.07,-13.28,FOOTWEAR,1.0,10184.51,X40N,10.49,53.40
355,VIPIND,307.35,718.00,-28.19,-729.00,H-SC,89.11,0.46,133.61,140.78,...,X40N,NTT,70.88,13.19,MISC,1.0,4607.81,X40N,5.25,-63.51
1,5PAISA,364.05,810.00,-19.99,123.56,H-SC,38.13,-1.99,122.50,102.18,...,X40N,BTT,39.09,9.46,FINANCE,1.0,1148.55,X40N,10.29,-85.27
158,INDIGOPNTS,988.25,2061.00,-25.97,86.94,H-SC,42.02,1.38,108.55,213.41,...,X40N,NTT,54.10,23.83,PAINTS,1.0,5165.56,X40N,17.52,61.95
172,JCHAC,1763.70,3542.00,-8.04,20117.39,M-SC,32.30,0.29,100.83,82.53,...,X40N,BTT,122.89,1.60,AC,1.0,4439.24,X40N,-7.81,2587.89
79,CERA,5350.50,10420.45,-27.23,73.94,X-SC,9.34,-0.12,94.76,94.76,...,X40N,ATH,41.90,21.49,CERAMICS,1.0,7277.21,X40N,19.05,64.28
113,FINCABLES,855.95,1657.02,-27.22,3.27,M-SC,11.28,-1.22,93.59,93.59,...,X40N,ATH,5.95,-3.79,CABLES,NaN,13284.34,X40N,14.20,73.39
333,TTKPRESTIG,611.60,1149.24,-24.12,-12.42,X-SC,23.74,0.83,87.91,87.91,...,X40N,ATH,0.80,-8.02,DURABLES,1.0,8398.33,X40N,11.19,76.01
323,TEAMLEASE,1863.40,3487.00,-30.03,147.79,M-SC,44.75,-1.44,87.13,183.39,...,X40N,NTT,50.68,14.55,MISC,NaN,3302.31,X40N,13.04,68.31


In [12]:
# xr, sr, ar
df_tmp = df_prospects_features[df_prospects_features['Criteria'].isin(['XR','SR', 'AR'])].sort_values(by = 'FTT%', ascending=False)
data_table.DataTable(df_tmp, include_index=False)

,Symbol,Close,FTT,Dev%_200,Dev%_PE,Conviction,RSP,Today%,FTT%,ATH%,...,Criteria,Strategy,Sales_Grwth%,Profit_Grwth%,Category,InFolio,MCap,MBQ,ROE%,CFO_2_EBITDA%
44,BAJAJHIND,19.63,442.93,-38.66,NaN,NaN,47.86,0.51,2156.39,2156.39,...,XR,ATH,NaN,NaN,SUGAR,NaN,NaN,XR,NaN,NaN
78,CENTRALBK,36.71,190.06,-30.74,-42.74,L-MC,40.08,-0.94,417.73,417.73,...,XR,ATH,9.56,46.67,BANKS,NaN,33571.84,XR,11.36,-9.91
15,ALOKINDS,16.42,79.81,-25.36,NaN,NaN,76.26,-0.97,386.05,386.05,...,AR,ATH,NaN,NaN,TEXTILES,NaN,NaN,AR,NaN,NaN
18,APEX,206.00,849.16,-14.88,NaN,NaN,14.40,-0.93,312.21,312.21,...,AR,ATH,NaN,NaN,MISC,NaN,NaN,AR,NaN,NaN
311,TANLA,498.55,1963.11,-28.68,-56.59,H-SC,73.15,5.10,293.76,293.76,...,AR,ATH,25.33,32.45,IT,1.0,6540.56,AR,31.71,75.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,INDIGO,5321.50,5558.50,17.03,44.69,M-LC,68.68,1.37,4.45,4.45,...,AR,ATH,17.22,-15.52,AIRLINES,NaN,202499.70,AR/X5K,NaN,119.94
209,MANAPPURAM,230.81,240.38,18.97,28.24,M-SC,31.52,-0.02,4.15,4.15,...,XR,ATH,21.39,-3.69,FINANCE,NaN,19548.37,XR,20.65,-55.72
127,GRASIM,2724.00,2832.86,4.77,143.83,H-LC,64.98,-0.49,4.00,4.00,...,AR,ATH,10.44,-27.65,TEXTILES,NaN,186209.71,AR/X5K,6.90,-37.81
98,DIVISLAB,6079.50,6256.50,8.49,39.44,X-LC,81.71,-0.12,2.91,2.91,...,AR,ATH,21.15,49.46,PHARMA,NaN,161487.62,AR,12.17,49.47
